I created a new kind of mpohamiltonian for quantum chemistry hamiltonians, because you can gain some speed there by pre-summing the environments. I also invested a bunch of effort making the effective hamiltonian fast. In this notebook I try to benchmark the MPSKit effective hamiltonians against my (?optimized) implementation.

I don't make use of the pre-summation in this benchmark, both codes should do the exact same thing.

In [1]:
using Revise, MPSKit, MPSKitModels, TensorKit
using LinearAlgebra, Base.Threads, BenchmarkTools
using MPSKitExperimental:FusedMPOHamiltonian

In [2]:
BLAS.set_num_threads(1)

In [3]:
nthreads()

4

## heisenberg spin 1

In [4]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

ts = FiniteMPS(rand,ComplexF64,fmps_len,Rep[SU₂](1=>1),Rep[SU₂](i => 20 for i in 0:10));

th_orig = heisenberg_XXX(SU2Irrep);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [5]:
@benchmark ac_eff_orig(ac)

BenchmarkTools.Trial: 3429 samples with 1 evaluation.
 Range (min … max):  835.424 μs … 13.222 ms  ┊ GC (min … max):  0.00% … 77.98%
 Time  (median):       1.178 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.451 ms ±  1.580 ms  ┊ GC (mean ± σ):  15.25% ± 12.38%

  █▇▅▂                                                          
  ████▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂ ▃
  835 μs          Histogram: frequency by time         11.4 ms <

 Memory estimate: 1.43 MiB, allocs estimate: 14411.

In [6]:
@benchmark ac_eff_fused(ac)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  33.590 μs …  15.737 ms  ┊ GC (min … max): 0.00% … 92.27%
 Time  (median):     84.420 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   98.488 μs ± 382.876 μs  ┊ GC (mean ± σ):  9.83% ±  2.53%

            ▅▇▇█▆▆▇▇▅▄▃▂▁                                       
  ▂▆▆▃▂▃▄▅▅▅██████████████▆▅▄▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  33.6 μs         Histogram: frequency by time          223 μs <

 Memory estimate: 123.69 KiB, allocs estimate: 971.

In [7]:
@benchmark ac2_eff_orig(ac2)

BenchmarkTools.Trial: 2475 samples with 1 evaluation.
 Range (min … max):  1.300 ms … 27.964 ms  ┊ GC (min … max):  0.00% … 85.46%
 Time  (median):     1.489 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   2.014 ms ±  2.939 ms  ┊ GC (mean ± σ):  20.46% ± 12.80%

  █▆▃                                                         
  ████▆▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▆▇ █
  1.3 ms       Histogram: log(frequency) by time     21.6 ms <

 Memory estimate: 3.55 MiB, allocs estimate: 35054.

In [8]:
@benchmark ac2_eff_fused(ac2)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  40.560 μs …  17.590 ms  ┊ GC (min … max): 0.00% … 94.11%
 Time  (median):     76.340 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   87.938 μs ± 290.214 μs  ┊ GC (mean ± σ):  5.53% ±  1.68%

                 ▃▆██▁                                          
  ▂▂▂▂▂▂▃▃▂▂▂▂▂▂▄█████▅▄▃▃▅▄▅▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  40.6 μs         Histogram: frequency by time          155 μs <

 Memory estimate: 75.88 KiB, allocs estimate: 1004.

## some weird hamiltonian

In [9]:
using MAT
include("haagerup/H3_anyon.jl")

In [10]:
fmps_len = 10;
middle_site = Int(round(fmps_len/2))

physical = Vect[H3](H3(4)=>1);
virtual = Vect[H3](sector => 20 for sector in TensorKit.SectorValues{H3}());
ts = FiniteMPS(rand,ComplexF64,fmps_len,physical,virtual);

mpotensor = TensorMap(ones,ComplexF64,physical*physical,physical*physical);
blocks(mpotensor)[H3(1)]*=0;
blocks(mpotensor)[H3(5)]*=0;
blocks(mpotensor)[H3(6)]*=0;
th_orig = MPOHamiltonian(-mpotensor);
th_fused = convert(FusedMPOHamiltonian,repeat(th_orig,length(ts)));

env_orig = environments(ts,th_orig);
env_fused = environments(ts,th_fused);

ac = ts.AC[middle_site]
ac_eff_orig = MPSKit.∂∂AC(middle_site,ts,th_orig,env_orig);
ac_eff_fused = MPSKit.∂∂AC(middle_site,ts,th_fused,env_fused);


ac2 = ts.AC[middle_site]*MPSKit._transpose_tail(ts.AR[middle_site+1])
ac2_eff_orig = MPSKit.∂∂AC2(middle_site,ts,th_orig,env_orig);
ac2_eff_fused = MPSKit.∂∂AC2(middle_site,ts,th_fused,env_fused);

In [11]:
@benchmark ac_eff_orig(ac)

BenchmarkTools.Trial: 859 samples with 1 evaluation.
 Range (min … max):  3.102 ms … 20.650 ms  ┊ GC (min … max):  0.00% … 70.13%
 Time  (median):     4.290 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   5.801 ms ±  4.297 ms  ┊ GC (mean ± σ):  25.62% ± 23.28%

   ▁▇██▅▂                                                     
  ▂██████▅▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▃▃▄▃▃▃▃▃ ▃
  3.1 ms         Histogram: frequency by time        18.4 ms <

 Memory estimate: 6.93 MiB, allocs estimate: 52568.

In [12]:
@benchmark ac_eff_fused(ac)

BenchmarkTools.Trial: 7634 samples with 1 evaluation.
 Range (min … max):  460.617 μs …  16.485 ms  ┊ GC (min … max): 0.00% … 94.52%
 Time  (median):     552.822 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   649.255 μs ± 663.427 μs  ┊ GC (mean ± σ):  4.59% ±  4.39%

   ▃▄▄▆██▇▄▂                                                     
  ▇██████████▇▆▅▄▃▃▃▃▃▂▂▂▂▂▂▂▂▃▄▄▆▆▇██▇▆▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁ ▃
  461 μs           Histogram: frequency by time          977 μs <

 Memory estimate: 405.88 KiB, allocs estimate: 1703.

In [13]:
@benchmark ac2_eff_orig(ac2)

BenchmarkTools.Trial: 279 samples with 1 evaluation.
 Range (min … max):  12.344 ms … 35.760 ms  ┊ GC (min … max):  0.00% … 58.52%
 Time  (median):     13.661 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   17.930 ms ±  8.132 ms  ┊ GC (mean ± σ):  23.56% ± 23.87%

   ▃█▅▂                                                        
  ▅████▅▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▂▂▂▃▅▃▄▄▃▃▃ ▂
  12.3 ms         Histogram: frequency by time        34.4 ms <

 Memory estimate: 33.43 MiB, allocs estimate: 324728.

In [14]:
@benchmark ac2_eff_fused(ac2)

BenchmarkTools.Trial: 5339 samples with 1 evaluation.
 Range (min … max):  802.875 μs …  16.076 ms  ┊ GC (min … max): 0.00% … 89.76%
 Time  (median):     849.284 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   931.388 μs ± 957.625 μs  ┊ GC (mean ± σ):  6.75% ±  6.17%

    ▃▆█▆▃▃▁▁                                                     
  ▃▇█████████▇▆▅▄▄▄▄▃▃▃▃▃▂▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂ ▃
  803 μs           Histogram: frequency by time         1.21 ms <

 Memory estimate: 377.53 KiB, allocs estimate: 4019.

## manueel

manually written out @tensor, compared to a version that used DelayedFact & friends. The whole recycling of intermediaries becomes mostly important when multithreading, so not very applicable here.

In [15]:
using MPSKitExperimental:DelayedFact,TransposeFact,free!

In [16]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)


    function tobench(x)
        @plansor y[-1 -2;-3] := le[-1 -2 5;2 3]*x[2 3;4]*r[4 5;-3]
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 5515 samples with 1 evaluation.
 Range (min … max):  540.607 μs … 14.691 ms  ┊ GC (min … max):  0.00% … 90.60%
 Time  (median):     637.016 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   899.579 μs ±  1.651 ms  ┊ GC (mean ± σ):  25.06% ± 12.71%

  █▆▂                                                          ▁
  ███▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇▇ █
  541 μs        Histogram: log(frequency) by time      12.3 ms <

 Memory estimate: 974.47 KiB, allocs estimate: 4095.

In [17]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)


    function tobench(x)

        lex = le*x
        lex_trans = transpose(lex,(1,2),(4,3))
        lex_trans*r
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 6529 samples with 1 evaluation.
 Range (min … max):  457.066 μs … 17.285 ms  ┊ GC (min … max):  0.00% … 94.75%
 Time  (median):     554.917 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   757.161 μs ±  1.434 ms  ┊ GC (mean ± σ):  23.12% ± 11.50%

  █▅▂                                                          ▁
  ███▇▄▃▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▅▅▆▇ █
  457 μs        Histogram: log(frequency) by time      11.6 ms <

 Memory estimate: 771.36 KiB, allocs estimate: 2589.

In [18]:
virtual = Rep[SU₂](i => 20 for i in 0:10);
physical = Rep[SU₂](1=>1)

ac = TensorMap(rand,ComplexF64,virtual*physical,virtual)

fun = let        
    e = transpose(TensorMap(rand,ComplexF64,oneunit(physical)*physical,physical*oneunit(physical)),(1,),(3,4,2))
    l = transpose(TensorMap(rand,ComplexF64,virtual*oneunit(physical)',virtual),(3,1),(2,))
    le = transpose(l*e,(2,5,4),(1,3))

    r = TensorMap(rand,ComplexF64,virtual*oneunit(physical),virtual)

    multfactory_lex = DelayedFact(codomain(le)←virtual,storagetype(r))
    transposefactor_lex = TransposeFact(codomain(le)←virtual,storagetype(r),(1,2),(4,3))
    multfact_out = DelayedFact(virtual*physical←virtual,storagetype(r))

    function tobench(x)
        t_1 = multfactory_lex()
        mul!(t_1,le,x)

        t_2 = transposefactor_lex(t_1)
        free!(multfactory_lex,t_1);

        y = multfact_out()

        mul!(y,t_2,r)

        free!(transposefactor_lex,t_2)
        y
    end
end

@benchmark fun(ac)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  235.498 μs …  14.324 ms  ┊ GC (min … max):  0.00% … 95.68%
 Time  (median):     254.949 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   311.675 μs ± 699.183 μs  ┊ GC (mean ± σ):  14.41% ±  6.27%

   ▃█▅▁                                                          
  ▂████▅▄▅▅▅▆▅▅▅▄▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  235 μs           Histogram: frequency by time          418 μs <

 Memory estimate: 198.03 KiB, allocs estimate: 227.